# Editor

[CodeMirror](https://github.com/codemirror/codemirror) is the editor that powers the Jupyter stack. `wxyz.lab.Editor` provides much of the configurability/observability of the underlying editor.

In [ ]:
from wxyz.lab import Editor, EditorModeInfo, DockPop

In [ ]:
import ipywidgets as W, traitlets as T, IPython as I, textwrap
from yaml import safe_dump

At its most basic, an `Editor` will only be large enough to contain its text.

In [ ]:
editor = Editor(value="hello: world", layout=dict(height="100%", width="100%"), 
                config=dict(mode="yaml", theme="vibrant-ink"))
W.HBox([editor], layout=dict(height="50px"))

In [ ]:
if __name__ == "__main__":
    I.display.display(DockPop([editor], mode="split-right"))

## Modes

JupyterLab makes all known (but not necessarily loaded) syntax _modes_. The `EditorModeInfo` widget makes this information available.

In [ ]:
mode_info = EditorModeInfo()
mode = W.SelectionSlider(options=["ipython"], description="mode", rows=1)
def mode_options(modes):
    modes = modes or [{"mime": "text/x-ipython", "mode": "ipython"}]
    return sorted(set(sum([[m.get("mime"), m.get("mode")] for m in modes], [])))

def link_modes(*change):
    editor.value += "\nmodes:\n" + textwrap.indent(safe_dump(mode_info.modes), "  ")
    T.dlink((mode_info, "modes"), (mode, "options"), mode_options)
    T.link((editor.config, "mode"), (mode, "value"))
mode_info.observe(link_modes, "modes")
mode

## Themes

CodeMirror uses CSS-based _themes_. They aren't particularly discoverable at runtime, but they usually don't get _removed_.

In [ ]:
THEMES = (
    "3024-day 3024-night abcdef ambiance ambiance-mobile ayu-dark ayu-mirage base16-dark base16-light "
    "bespin blackboard cobalt colorforth darcula dracula duotone-dark duotone-light default eclipse elegant "
    "erlang-dark gruvbox-dark hopscotch icecoder idea isotope lesser-dark liquibyte lucario material "
    "material-darker material-ocean material-palenight mbo mdn-like midnight monokai moxer neat neo night "
    "nord oceanic-next panda-syntax paraiso-dark paraiso-light pastel-on-dark railscasts rubyblue seti "
    "shadowfox solarized ssms the-matrix tomorrow-night-bright tomorrow-night-eighties ttcn twilight "
    "vibrant-ink xq-dark xq-light yeti yonce zenburn"
)
editor.value += "\n" + f"themes: {THEMES}"
theme = W.SelectionSlider(options=THEMES.split(), description="theme")
T.link((editor.config, "theme"), (theme, "value"))
theme

## Scrolling
The top-left position of the editor is readable and writeable from `scroll_x` and `scroll_y`. These are, unfortunately, in _pixels_, not _characters_, as other settings like `lineWrapping`

In [ ]:
scroll_x = W.IntSlider(description="x", max=2000); W.jslink((scroll_x, "value"), (editor, "scroll_x"))
scroll_y = W.IntSlider(description="y", max=2000); W.jslink((scroll_y, "value"), (editor, "scroll_y"))
wrap = W.Checkbox(description="lineWrapping"); W.jslink((wrap, "value"), (editor.config, "lineWrapping"));

In [ ]:
if __name__ == "__main__":
    I.display.display(scroll_x, scroll_y, wrap)

Complicating matters even further are inline widgets, like code folding.

In [ ]:
fold = W.Checkbox(description="foldGutter"); T.link((fold, "value"), (editor.config, "foldGutter"))
line_numbers = W.Checkbox(description="lineNumbers"); T.link((line_numbers, "value"), (editor.config, "lineNumbers"))
gutters = W.SelectMultiple(
    options=["CodeMirror-linenumbers", "CodeMirror-foldgutter"], 
    description="gutters"
); T.link((gutters, "value"), (editor.config, "gutters"));

In [ ]:
if __name__ == "__main__":
    I.display.display(line_numbers, fold, gutters) 

## The App

In [ ]:
editor.layout.max_width = "70vw"
app = W.HBox(
    [
        W.VBox([mode, theme, wrap, scroll_x, scroll_y, line_numbers, fold, gutters], 
               layout=dict(min_width="300px")), 
        editor
    ], 
    layout=dict(width="100%", height="600px", display="flex", max_height="100%")
)

In [ ]:
if __name__ == "__main__":
    I.display.display(app)

In [ ]:
if __name__ == "__main__":
    with __import__("importnb").Notebook():
        from wxyz.notebooks import Utils
        Utils.maybe_log_widget_counts()